In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib
matplotlib.pyplot.style.use('seaborn')
matplotlib.rcParams['figure.figsize'] = (15, 5)

%matplotlib inline

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
np.set_printoptions(precision=2, suppress=True)

In [ ]:
import math
import copy

import scipy.stats as stats

In [ ]:
from sklearn import model_selection, metrics, datasets

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
np.random.seed(42)

FA tries to explain the dependencies among the observed variables in terms of a smaller number of unobserved latent variables, or factors, without loss of information. Statistically, the dependencies between variables are defined by the covariance.

# Model

_(Attention: not an i'th observation, but a i'th feature in theoretical vector)_

* $i=0..p$, $j=0..k$

* $k < p$

$\vec{x}$ - some, _theoretical_ vector of featues. $x \in X$, $\mathbb{R}^p$, _(normalized: $x = x - \bar{x}$)_

$\vec{f}$ -  generalizing factors, explaining `x`. $f \in F$, $\mathbb{R}^k$

$\vec{u}$ - uniqueness of feature, "specific variates". $u \in U$, $\mathbb{R}^p$

$A_{i,j}$ - factor loadings

---

$x_i - \bar{x}_i = (a_{i,0} f_0 + a_{i,1} f_1 + ... + a_{i,k} f_k) + u_i$

$X - \mu = AF + U$

$\begin{bmatrix}
x_{0, 0} & x_{0, 1} & ... & x_{0, p} \\
... \\
x_{n, 0} & x_{n, 1} & ... & x_{n, p}
\end{bmatrix} = $
$\left(\begin{bmatrix}
a_{0,0} & a_{0,1} & ... & a_{0,k} \\
... \\
a_{p,0} & a_{p,1} & ... & a_{p,k}
\end{bmatrix} \cdot
\begin{bmatrix}
f_{0,0} & f_{0,1} & ... & f_{0,k} \\
... \\
f_{n,0} & f_{n,1} & ... & f_{n,k} \\
\end{bmatrix}^T\right)^T + U + \mu$

In [ ]:
P = 9
K = 5

assert K < P

N = 3

# ------------------------------

F = np.ones((N, K))
F

U = np.random.randn(N, P)

A = np.random.randn(P, K) # i=P, j = K

X = A.dot(F.T).T + U
X

### Restirctions:

$E(X) = 0$

$E(F) = 0$

$D(F) = I$ - Each factor explains its own characteristics. Variance of factors equals $1$

$U$ and $F$ independent _(Soft: no correlation)_

In [ ]:
X = np.array([
    [3.,6.,5.],
    [ 7, 3, 3],
    [10, 9, 8],
    [ 3, 9, 7],
    [10, 6, 5],
])

S = np.cov(X, rowvar=False, bias=True,)
S2 = ((X - X.mean(axis=0)).T.dot((X - X.mean(axis=0)))) / len(X)
assert np.isclose(S, S2, 1e-11).all()

# Theoretical variance-covariance matrix 

$D(X_i) = A^2_{i, 0}\cdot D(F_0)
+ A^2_{i, 1}\cdot D(F_1)
+ ... 
+ A^2_{i, k}\cdot D(F_k)
+ (1)^2D(u_i)$

---

because $D(F) = I$:

$D(X_i) = A^2_{i, 0}
+ A^2_{i, 1}
+ ... A^2_{i, k}
+ (1)^2D(u_i)$

$D(X_i) = \sum_j^k (A_{i,j})^2 + D(u_i)$

$h^2_j = \sum_j^k (A_{i,j})^2$ - communality

$D(u_i)$ - unique variance

# * * *

$cov(x_i, x_j) = A_{i, 0}\cdot A_{j, 0} \cdot D(F_0)
+ A_{i, 1}\cdot A_{j, 1} \cdot D(F_1)
+ ...
+ A_{i, p}\cdot A_{j, k} \cdot D(F_k)$

---

because $D(F) = I$:

$cov(x_i, x_j) = A_{i, 0}\cdot A_{j, 0}
+ A_{i, 1}\cdot A_{j, 1}
+ ...
+ A_{i, p}\cdot A_{j, k}$

$cov(x_i, x_j) =  \sum_m^k a_{i,m} a_{j,m}$

---

Summary:

$\Sigma = AA^T + diag(D(U))$, Where dioganal $h^2_j$ - see abowe

---

### Proof

Note $D = Cov$

Then note that from the conditions just imposed on $A$, we have:

$Cov(X - \mu) = Cov(AF + U)$

$\Sigma = ACov(F)A^T + Cov(U)$

---

* $Cov(F) = I$
* $\Psi = diag(Cov(U))$

$\rightarrow$

$\Sigma = AA^T +\Psi$

# * * *

Task: $\left\| \Sigma - AA^T \right\| \rightarrow min$

# TODO: how?

* Generate $F$ with restrictions abowe

* Solve for $X - \mu = AF + U$ with one unknown - $A$

* Set of all posible equation reduce to answer by varimax

$\Sigma = Cov(X)$

$\hat{\Sigma} = AA^T = Cov(AF)$

1) перебор значений $A$, $F$

2) подсчёт $\hat{\Sigma} = Cov(AF)$

3) расчёт $\left\| \Sigma - \hat{\Sigma} \right\|^2$ (или другой нормы)

Накладываем требования на $F$: $Cov(F) = I$

Накладываем требования на $A$: varimax 